# Cloud Pak for Data Dashboard - Server Status
This notebook can be used to update the server status display.

**Note:** The regular dashboard service uses port 7272, while this one uses 7273. Both dashboards use identical API calls but the objects (tiles) that they refer to are different. Make sure you use port 7273 to update the proper display.

In [ ]:
%run dashboard.ipynb
seturl('http://172.17.0.3:7272')

There are 8 slots available for status information. Each tile has a name (server1..8) which are spread over two rows on the screen. Server1..4 are on the top row and server5..8 are on the bottom row.

The `publishJustValue` function requires 5 arguments:

* Tile name - server1 ... server8
* Server name - Any string of characters that describe the server
* Description - This could be the IP address of the server or any other key information about the service
* Status - A string that represents up time or whether the server is down or not
* Color - The background color of the status box. If the color is red, then the box will flash.

Valid colors are blue, green, red, yellow, white, black, violet, orange, naval, or tile_background. The tile_backgound color blends in with the form so it is the equivalent of no color.

If you do not provide a value for a server tile, it will retain the previous value that was supplied. In other words, if you had provided values for servers 1-4, the values 5-8 would remain blank. If you update server 1, the other tiles will still contain the old values. If you want to delete any tiles you must provide blank values to remove them (and a color of tile_background).

In [ ]:
!wget -O db2.ipynb https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/db2.ipynb 
%run db2.ipynb

In [ ]:
%run connection.ipynb

In [ ]:
def connectionAvailableTitle(CPOOLSIZE):
    if CPOOLSIZE == '0':
        return 'Down'
    else:
        return 'Up'  

def connectionAvailableColor(CPOOLSIZE):
    if CPOOLSIZE == '0':
        return 'red'
    else:
        return 'green'

In [ ]:
def getConnectionStatus():
    virtualizationConnections = %sql SELECT CID, USR, SRCTYPE, SRCHOSTNAME, SRCPORT, DBNAME FROM QPLEXSYS.LISTRDBC ORDER BY CID
    connectionStatus = %sql call QPLEXSYS.LISTRDBCDETAILS()

    # Join the DV connections to the CPOOLSIZE
    arrayPoolSize = []
    for index, row in virtualizationConnections.iterrows():
        cid = row['CID']
        cidToCheck = connectionStatus.loc[connectionStatus['CID'] == cid]
        arrayPoolSize.append(cidToCheck['CPOOLSIZE'].iat[0])

    server = ['server1','server2','server3','server4','server5','server6','server7','server8']
    virtualizationConnections['CPOOLSIZE'] = arrayPoolSize
    virtualizationConnections['STATUS'] = arrayPoolSize
    virtualizationConnections['COLOR'] = arrayPoolSize

    virtualizationConnections['STATUS'] = virtualizationConnections['STATUS'].apply(connectionAvailableTitle)
    virtualizationConnections['COLOR'] = virtualizationConnections['COLOR'].apply(connectionAvailableColor)

    cs = virtualizationConnections[['SRCTYPE','SRCHOSTNAME','STATUS','COLOR']]
    cs.rename(columns={'SRCTYPE':'TITLE'}, inplace=True)
    cs.rename(columns={'SRCHOSTNAME':'DESCRIPTION'}, inplace=True)
    cs.rename(columns={'STATUS':'VALUE'}, inplace=True)
    cs.rename(columns={'COLOR':'COLOR'}, inplace=True)
    return cs

In [ ]:
while True: 
    cs = getConnectionStatus()
    publishJustValue("server1","AWS Db2",cs.iloc[2]['DESCRIPTION'],cs.iloc[2]['VALUE'],cs.iloc[2]['COLOR'])
    publishJustValue("server2","Azure Db2",cs.iloc[3]['DESCRIPTION'],cs.iloc[3]['VALUE'],cs.iloc[3]['COLOR'])
    publishJustValue("server3","Azure Postgres",cs.iloc[7]['DESCRIPTION'],cs.iloc[7]['VALUE'],cs.iloc[7]['COLOR'])
    publishJustValue("server4","MongoDB",cs.iloc[6]['DESCRIPTION'],cs.iloc[6]['VALUE'],cs.iloc[6]['COLOR'])
    publishJustValue("server5","Informix",cs.iloc[5]['DESCRIPTION'],cs.iloc[5]['VALUE'],cs.iloc[5]['COLOR'])
    publishJustValue("server6","Db2 Warehouse",cs.iloc[0]['DESCRIPTION'],cs.iloc[0]['VALUE'],cs.iloc[0]['COLOR'])
    publishJustValue("server7","z/OS",cs.iloc[4]['DESCRIPTION'],cs.iloc[4]['VALUE'],cs.iloc[4]['COLOR'])
    publishJustValue("server8","Db2",cs.iloc[1]['DESCRIPTION'],cs.iloc[1]['VALUE'],cs.iloc[1]['COLOR'])
    
    time.sleep(60)
